In [17]:
# Imports
import keras
from keras.preprocessing.image import ImageDataGenerator
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import os, glob, random, sys, math, cv2
from sklearn.model_selection import train_test_split
from skimage import transform
from tqdm import tqdm

# Model
from model import Model

%matplotlib inline

In [7]:
# Read image categories
ROOT_DIR = os.getcwd()
IMAGE_DIR = os.path.join(ROOT_DIR, 'images')
CATEGORIES = [folder for folder in os.listdir(IMAGE_DIR) 
               if os.path.isdir(os.path.join(IMAGE_DIR, folder))]
NUM_CLASSES = len(CATEGORIES)
print(CATEGORIES)

['orchid']


In [14]:
IMAGE_DIM = (512, 512,)

# Read images and labels into arrays
images = []
labels = []
for label, category in enumerate(CATEGORIES):
    print("Processing {} images...".format(category))
    folder_path = os.path.join(IMAGE_DIR, category)
    # Read images in subfolder
    for image_file_path in tqdm(glob.glob(folder_path + "/*.jpg")):
        image_data = cv2.imread(image_file_path)
        # Resize to constant dimensions
        image_data = transform.resize(image_data, IMAGE_DIM)
        images.append(image_data)
        labels.append(label)

print("Read {} images and {} labels.".format(len(images), len(labels))) # Should be same number

0it [00:00, ?it/s]

Processing orchid images...
Read 0 images and 0 labels.


In [18]:
# Split into training and test sets
train_test_split_ratio = 0.9 # Proportion in train set
(x_train, y_train), (x_test, y_test) = train_test_split(images, labels, train_size = train_test_split_ratio)
print(x_train.shape())
print("{} training images, {} testing images".format(len(x_train, y_train)))

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [ ]:
# Image data augmentation to increase effective dataset size
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255, # Rescale to range 0..1
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='wrap', # I.e. tiling
)

In [ ]:
# Model
model = Model(IMAGE_DIM + (3,)) # 3 channels
model_checkpoint = keras.ModelCheckpoint('weights.{epoch:02d}.h5', verbose=1)

# Possibly load trained weights here

In [ ]:
# Train model

# Hyper parameters
BATCH_SIZE = 8
EPOCHS = 100
STEPS_PER_EPOCH = len(x_train) // BATCH_SIZE

model.fit_generator(datagen.flow(x_train, y_train, batch),
                    steps_per_epoch = STEPS_PER_EPOCH,
                    epochs = EPOCHS,
                    callbacks = [model_checkpoint],
                   )